In [ ]:
# python version36에서 해야함

from imutils import face_utils
import numpy as np
import imutils
import dlib
import cv2
import matplotlib.pyplot as plt
import face_recognition
import os
from imutils import paths

In [ ]:
def imshow(tit, image) :
    plt.title(tit)    
    if len(image.shape) == 3 :
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    else :
        plt.imshow(image, cmap="gray")
    plt.show()

In [ ]:
files = os.listdir("known") #아는 얼굴을 학습
    
known_face_encodings = []  #다음 for문을 통해 128개의 실수값이 들어가게된다.
known_face_names = []

for filename in files:
            name, ext = os.path.splitext(filename) # 검출해서 어떤 이미지와 유사한지 이름 알려주기 위해
            if ext == '.jpg':
                known_face_names.append(name)
                pathname = os.path.join("known", filename)
                img = face_recognition.load_image_file(pathname) #이거때문에 한사람의 얼굴만 있다고 가정해야함
                face_encoding = face_recognition.face_encodings(img)[0] #발견된 얼굴 중 0번째
                known_face_encodings.append(face_encoding)

In [ ]:
print(known_face_names) # 파일들이 갖는 이름들 출력
print(known_face_encodings[1]) # 0번 파일의 128차원 값

In [ ]:
# 테스트 이미지
test = face_recognition.load_image_file("facetest.jpg")

face_locations = face_recognition.face_locations(test)
face_encodings = face_recognition.face_encodings(test, face_locations)

face_names = []
pre_name = "Unknown"

for face_encoding in face_encodings: # face마다 거리 측정
     distances = face_recognition.face_distance(known_face_encodings, face_encoding)
     print(distances) # 거리 값 확인
     min_value = min(distances)  

     #pre_name = "Unknown"
     if min_value < 0.6: # 거리에 대한 최대 값 지정(0.6보다 크면 다른 사람 일 확률이 크다는 의미)
                    index = np.argmin(distances)
                    name = known_face_names[index] # 검출하면 이름 저장
print(name)

In [ ]:
imagePaths = list(paths.list_images("known")) # 폴더 경로
 
knownEncodings = []
knownNames = []

for (i, imagePath) in enumerate(imagePaths): # 이미지 하나씩 가져와서 얼굴 검출
    name = imagePath.split(os.path.sep)[-2] # 가져오는 데이터 중에 끝에서 두번째 것(폴더명)
    print(f"{name}   -   {imagePath}") # 이미지파일 이름과 경로 확인용
    image = cv2.imread(imagePath) # 이미지 저장해서
    boxes = face_recognition.face_locations(image) # 얼굴 검출 좌표 값
    encodings = face_recognition.face_encodings(image, boxes) # 얼굴 검출 정보 값

    for encoding in encodings: # 이미지 중 얼굴이 검출되었다면, for loop를 수행
        knownEncodings.append(encoding)
        knownNames.append(name)

In [ ]:
# 파일로 저장 & 불러오기

import pickle
# 일반 텍스트 파일은 파일 입출력을 이용해서 불러올 수 있지만
# 리스트, 클래스 등의 자료형 데이터를 저장하거나 불러올 수 없기 때문에
# pickle 모듈을 사용해서 불러와야 한다.

data = {"encodings": knownEncodings, "names": knownNames} #딕셔너리로 넣고 딕셔너리로 나중에 읽음
f = open("known.bin", "wb") # bin 파일을 연다.(name data와 인코딩한 데이터가 담긴 정보) 
                        #wb: write binary
f.write(pickle.dumps(data)) # 입력
f.close()

In [ ]:
# 위 코드 확인용

print(data["name"]) # 얼굴 검출 한 이미지들의 폴더명(like 클래스명)

print(data["name"][0]) # 이름의 0번 인덱스 값 가져와서 확인
print(data["encodings"][0]) # 0번 이미지의 얼굴 정보에 대한 값 확인

In [ ]:
data = pickle.loads(open("known.bin", "rb").read())
 
image = cv2.imread("songtest.jpg") # 테스트 할 이미지

boxes = face_recognition.face_locations(image) # 테스트 할 이미지의 얼굴 영역 검출
encodings = face_recognition.face_encodings(image, boxes) # 얼굴 영역에 대한 정보 저장
 
names = []
for encoding in encodings: # 폴더 속 이미지들을 순회하며 유사 얼굴 탐색
    matches = face_recognition.compare_faces(data["encodings"], encoding)
    name = "Unknown" # 매칭되는 얼굴이 없는 경우
    
    if True in matches:
        matchedIdxs = [i for (i, b) in enumerate(matches) if b]
        counts = {} 
        
        for i in matchedIdxs: # True 판단 이미지(유사하다고 추정된 이미지)만 탐색하면서
            name = data["names"][i] # 파일명 가져오고
            counts[name] = counts.get(name, 0) + 1  # name을 count
        name = max(counts, key=counts.get) # 검출 된 이름의 빈도수에서 가장 큰 값을 결과로
    names.append(name) # 결과 name 저장
    

In [ ]:
# 찾은 수 만큼 사각형 그리고 이름 적기
for ((top, right, bottom, left), name) in zip(boxes, names):
    cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
    y = top - 15 if top - 15 > 15 else top + 15
    cv2.putText(image, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX,  0.75, (0, 255, 0), 2)
 
imshow("Image", image)